## Deliverable 2. Create a Customer Travel Destinations Map.

In [28]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [29]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Sisimiut,GL,66.9395,-53.6735,31.42,84,100,33.80,snow
1,1,Tasiilaq,GL,65.6145,-37.6368,21.36,61,100,3.02,overcast clouds
2,2,Lawton,US,34.6087,-98.3903,64.87,32,1,13.80,clear sky
3,3,Saint George,US,37.1041,-113.5841,59.34,27,12,1.01,few clouds
4,4,Ribeira Grande,PT,38.5167,-28.7000,67.86,82,100,20.20,overcast clouds


In [31]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 57
What is the maximum temperature you would like for your trip? 76


In [32]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
2,2,Lawton,US,34.6087,-98.3903,64.87,32,1,13.80,clear sky
3,3,Saint George,US,37.1041,-113.5841,59.34,27,12,1.01,few clouds
4,4,Ribeira Grande,PT,38.5167,-28.7000,67.86,82,100,20.20,overcast clouds
8,8,Rodrigues Alves,BR,-7.7380,-72.6509,73.81,92,66,1.70,broken clouds
10,10,Puerto Ayora,EC,-0.7393,-90.3518,69.19,80,4,7.36,clear sky
15,15,Rio Grande,BR,-32.0350,-52.0986,60.58,68,0,8.34,clear sky
16,16,Mar Del Plata,AR,-38.0023,-57.5575,69.82,49,0,4.00,clear sky
18,18,Santa Maria,BR,-29.6842,-53.8069,67.68,54,0,9.93,clear sky
19,19,Victoria,HK,22.2855,114.1577,65.32,67,0,4.00,clear sky
20,20,Nouadhibou,MR,20.9310,-17.0347,71.69,72,100,14.00,overcast clouds


In [33]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID        297
City           297
Country        297
Lat            297
Lng            297
Max Temp       297
Humidity       297
Cloudiness     297
Wind Speed     297
Description    297
dtype: int64

In [34]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [40]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Description", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Description,Max Temp,Lat,Lng,Hotel Name
2,Lawton,US,clear sky,64.87,34.6087,-98.3903,
3,Saint George,US,few clouds,59.34,37.1041,-113.5841,
4,Ribeira Grande,PT,overcast clouds,67.86,38.5167,-28.7000,
8,Rodrigues Alves,BR,broken clouds,73.81,-7.7380,-72.6509,
10,Puerto Ayora,EC,clear sky,69.19,-0.7393,-90.3518,
15,Rio Grande,BR,clear sky,60.58,-32.0350,-52.0986,
16,Mar Del Plata,AR,clear sky,69.82,-38.0023,-57.5575,
18,Santa Maria,BR,clear sky,67.68,-29.6842,-53.8069,
19,Victoria,HK,clear sky,65.32,22.2855,114.1577,
20,Nouadhibou,MR,overcast clouds,71.69,20.9310,-17.0347,


In [41]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping...")
        

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [42]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [43]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [52]:
hotel_df.head(10)

,City,Country,Description,Max Temp,Lat,Lng,Hotel Name
2,Lawton,US,clear sky,64.87,34.6087,-98.3903,Holiday Inn Express & Suites Lawton-Fort Sill
3,Saint George,US,few clouds,59.34,37.1041,-113.5841,Best Western Plus Abbey Inn
4,Ribeira Grande,PT,overcast clouds,67.86,38.5167,-28.7000,Quinta da Meia Eira
8,Rodrigues Alves,BR,broken clouds,73.81,-7.7380,-72.6509,Pousada Aconchego
10,Puerto Ayora,EC,clear sky,69.19,-0.7393,-90.3518,Finch Bay Galapagos Hotel
15,Rio Grande,BR,clear sky,60.58,-32.0350,-52.0986,Hotel Atlântico Rio Grande
16,Mar Del Plata,AR,clear sky,69.82,-38.0023,-57.5575,Gran Hotel Mar del Plata
18,Santa Maria,BR,clear sky,67.68,-29.6842,-53.8069,Hotel Morotin
19,Victoria,HK,clear sky,65.32,22.2855,114.1577,Mini Hotel Central
20,Nouadhibou,MR,overcast clouds,71.69,20.9310,-17.0347,El Medina


In [50]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [51]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure

# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [53]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))